In [1]:
import json, math, lkh, requests
from pathlib import Path
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pygad
import numpy as np
from python_tsp.heuristics import solve_tsp_local_search, solve_tsp_simulated_annealing
from python_tsp.exact import solve_tsp_dynamic_programming

def euclidean_3d(a, b):
    return math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2 + (a[2]-b[2])**2)

def tour_length(tour, pts):
    return sum(euclidean_3d(pts[tour[i]], pts[tour[i-1]]) for i in range(len(tour)))

def two_opt(pts, max_iter=10000):
    n = len(pts)
    tour = list(range(n))
    random.shuffle(tour)
    best_length = tour_length(tour, pts)
    improved = True
    iter_count = 0
    while improved and iter_count < max_iter:
        improved = False
        for i in range(n - 1):
            for j in range(i + 2, n):
                if j == n - 1 and i == 0:
                    continue
                new_tour = tour[:i+1] + tour[i+1:j+1][::-1] + tour[j+1:]
                new_length = tour_length(new_tour, pts)
                if new_length < best_length:
                    tour = new_tour
                    best_length = new_length
                    improved = True
        iter_count += 1
    return tour

In [2]:

# Read marker data from JSON file
mrk_path = Path("UTF-8ScannedPoints.mrk.json")
with mrk_path.open() as f:
    data = json.load(f)
    if data:
        print("Successfully loaded marker data")
    else:
        print("Failed to load marker data")
        exit(1)
        
# Points are in markups[0].controlPoints[*].position = [x,y,z]
pts = [tuple(cp["position"]) for cp in data["markups"][0]["controlPoints"]]
print(f"Loaded {len(pts)} points from {mrk_path}")

# Build distance matrix
n = len(pts)
dist_matrix = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        dist_matrix[i, j] = euclidean_3d(pts[i], pts[j])


Successfully loaded marker data
Loaded 357 points from UTF-8ScannedPoints.mrk.json


In [3]:
with open("scan3d.tsp", "w") as f:
    f.write(f"NAME: scan3d\nTYPE: TSP\nDIMENSION: {len(pts)}\n")
    f.write("EDGE_WEIGHT_TYPE: EUC_3D\n")
    f.write("NODE_COORD_SECTION\n")
    for i, (x,y,z) in enumerate(pts, start=1):
        f.write(f"{i} {x} {y} {z}\n")
    f.write("EOF\n")


In [4]:
tours = lkh.solve(
    solver="/usr/local/bin/LKH",    # or just "LKH" if on PATH",
    problem_file="scan3d.tsp",      # TSPLIB file we wrote above",
    output_tour_file="scan3d.tour",
    )

# Print the best tour found, and tour length
tour_pts = [pts[i-1] for i in tours[0]]  # LKH returns 1-based indices
print("LKH Tour length:", tour_length([i-1 for i in tours[0]], pts))
print("LKH Tour:", [i-1 for i in tours[0]])

LKH Tour length: 697.6702134005326
LKH Tour: [0, 1, 2, 10, 9, 21, 37, 57, 56, 36, 19, 20, 7, 8, 22, 23, 39, 38, 58, 78, 77, 96, 76, 95, 94, 75, 55, 54, 74, 73, 92, 91, 72, 52, 53, 35, 34, 18, 33, 51, 32, 17, 16, 31, 49, 68, 69, 50, 70, 71, 89, 109, 129, 108, 88, 87, 107, 128, 151, 173, 196, 218, 237, 217, 216, 236, 258, 238, 239, 259, 281, 298, 314, 300, 283, 299, 315, 330, 344, 355, 354, 353, 343, 352, 341, 351, 342, 329, 316, 317, 318, 331, 332, 345, 346, 333, 320, 334, 319, 304, 303, 288, 287, 302, 301, 286, 266, 246, 227, 207, 229, 250, 231, 209, 210, 230, 208, 185, 186, 187, 164, 163, 162, 141, 120, 101, 83, 64, 84, 66, 65, 46, 45, 44, 63, 82, 62, 80, 60, 61, 79, 99, 81, 100, 119, 139, 140, 118, 98, 97, 117, 138, 116, 136, 157, 158, 179, 180, 202, 223, 243, 222, 242, 263, 241, 220, 200, 201, 221, 199, 178, 155, 133, 132, 154, 175, 152, 131, 153, 130, 110, 90, 111, 112, 93, 113, 114, 115, 135, 134, 156, 177, 176, 198, 219, 197, 174, 195, 194, 171, 149, 150, 172, 193, 192, 213, 214,

In [5]:
# 2-opt heuristic
two_opt_tour = two_opt(pts)

print("2-opt tour:", two_opt_tour)
print("2-opt length:", tour_length(two_opt_tour, pts))

2-opt tour: [120, 121, 102, 103, 84, 85, 66, 65, 45, 28, 29, 30, 46, 47, 48, 67, 86, 106, 127, 126, 125, 145, 168, 146, 147, 148, 170, 169, 191, 190, 212, 211, 232, 252, 273, 272, 251, 271, 250, 230, 231, 210, 209, 188, 189, 167, 166, 165, 143, 144, 124, 105, 104, 123, 122, 142, 164, 185, 186, 187, 208, 229, 249, 270, 269, 290, 268, 267, 288, 289, 305, 304, 319, 320, 334, 333, 332, 331, 330, 343, 344, 345, 346, 355, 354, 353, 352, 342, 341, 328, 329, 316, 301, 285, 284, 299, 300, 315, 314, 327, 340, 339, 338, 349, 350, 351, 356, 348, 347, 337, 325, 326, 313, 298, 297, 312, 296, 295, 294, 309, 310, 311, 324, 323, 336, 335, 322, 321, 307, 306, 291, 292, 308, 293, 277, 276, 275, 274, 253, 254, 233, 255, 256, 234, 214, 213, 192, 193, 172, 171, 149, 150, 128, 107, 108, 87, 88, 69, 68, 49, 31, 50, 70, 51, 32, 16, 17, 18, 34, 33, 52, 53, 73, 54, 35, 36, 20, 19, 7, 8, 21, 37, 56, 55, 74, 92, 112, 133, 132, 155, 177, 178, 200, 220, 240, 219, 198, 199, 176, 175, 197, 196, 174, 152, 153, 154, 131

In [6]:
# TSP solver using python-tsp (local search, simulated annealing, and dynamic programming)

# Simulated annealing (metaheuristic alternative)
perm_sa, length_sa = solve_tsp_simulated_annealing(dist_matrix)
print("python-tsp simulated annealing tour:", perm_sa)
print("python-tsp simulated annealing length:", length_sa)

# Local search (fast, good quality)
perm_ls, length_ls = solve_tsp_local_search(dist_matrix, x0=perm_sa, perturbation_scheme="ps3")
print("python-tsp local search tour:", perm_ls)
print("python-tsp local search length:", length_ls)


python-tsp simulated annealing tour: [0, 2, 10, 11, 23, 24, 40, 39, 38, 21, 22, 9, 1, 8, 7, 20, 19, 35, 36, 37, 56, 55, 54, 73, 72, 53, 52, 34, 18, 17, 16, 32, 33, 51, 70, 89, 71, 90, 91, 92, 112, 133, 134, 155, 132, 111, 110, 131, 130, 109, 108, 88, 69, 50, 31, 49, 68, 87, 107, 128, 149, 150, 129, 152, 151, 173, 172, 171, 192, 213, 214, 233, 254, 253, 274, 275, 255, 234, 215, 193, 194, 195, 216, 217, 196, 197, 175, 174, 153, 154, 176, 198, 199, 177, 200, 178, 156, 179, 157, 135, 136, 158, 180, 202, 203, 224, 223, 201, 222, 221, 220, 219, 218, 238, 239, 240, 241, 242, 243, 264, 265, 266, 244, 225, 245, 246, 226, 227, 228, 207, 206, 205, 204, 182, 181, 159, 137, 138, 117, 116, 115, 114, 113, 93, 74, 75, 94, 95, 77, 96, 76, 57, 58, 59, 60, 61, 62, 63, 64, 82, 81, 99, 80, 79, 78, 97, 98, 119, 118, 139, 140, 161, 160, 183, 184, 185, 186, 187, 208, 229, 249, 248, 247, 268, 289, 304, 288, 267, 287, 303, 318, 302, 286, 285, 300, 299, 283, 298, 313, 326, 327, 340, 341, 328, 314, 315, 329, 330,

In [ ]:
# Visualize point cloud, and path in 3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
xs, ys, zs = zip(*pts)
ax.scatter(xs, ys, zs, c='b', marker='o')
tour = tours[0]
tour_pts = [pts[i-1] for i in tour]  # tour is 1-based
txs, tys, tzs = zip(*tour_pts)
ax.plot(txs, tys, tzs, c='r')
plt.show()